In [395]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [396]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score
import lightgbm as lgb
import catboost as cb
import xgboost as xgb
import optuna
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

In [397]:
dataset_df = pd.read_csv('train.csv')
dataset_df['EJ'] = dataset_df['EJ'].replace({'A':0, 'B':1})
dataset_df = dataset_df.iloc[:,1:]

In [398]:

# feature_columns = dataset_df.columns[:-1]  # 假设最后一列为目标变量，不需要处理

# # 针对每个特征列，将无穷大值替换为前 97% 大的值
# for column in feature_columns:
#     # 计算前 97% 大的值
#     percentile = 0.98
#     threshold = dataset_df[column].quantile(percentile)
    
#     # 将无穷大值替换为前 97% 大的值
#     dataset_df[column] = dataset_df[column].replace(np.inf, threshold)

In [428]:
# 计算特征的均值、方差、标准差
mean_values = dataset_df.mean()
var_values = dataset_df.var()
std_values = dataset_df.std()
# 填充缺失值为特征的均值
dataset_df = dataset_df.fillna(mean_values)
max_values = dataset_df.max()
min_values = dataset_df.min()

In [400]:
# 分割特征和类别
X = dataset_df.iloc[:, :-1]  # 特征
y = dataset_df.iloc[:, -1]   # 类别
# X = X.dropna()  # 删除包含缺失值的行
y = y[X.index]  # 保持与特征对应的类别

# 对非数值类型特征进行独热编码
X_encoded = pd.get_dummies(X)

# 初始化特征选择器
k = 50  # 选择前k个重要特征
selector = SelectKBest(score_func=mutual_info_classif, k=k)

# 特征选择
X_selected = selector.fit_transform(X_encoded, y)

# 获取选择的特征索引
selected_feature_indices = selector.get_support(indices=True)

# 获取选择的特征名称
selected_features = X_encoded.columns[selected_feature_indices]
# selected_features = selected_features[:-2].append(pd.Index(['EJ']))
# 输出选择的特征
print(selected_features)

Index(['AB', 'AF', 'AH', 'AM', 'AX', 'BC', 'BD ', 'BN', 'BP', 'BQ', 'BR', 'CC',
       'CD ', 'CF', 'CL', 'CR', 'CS', 'CU', 'CW ', 'DA', 'DE', 'DF', 'DH',
       'DI', 'DL', 'DN', 'DU', 'DV', 'DY', 'EB', 'EE', 'EG', 'EH', 'EJ', 'EL',
       'EP', 'EU', 'FC', 'FD ', 'FE', 'FI', 'FL', 'FR', 'FS', 'GB', 'GE', 'GF',
       'GH', 'GI', 'GL'],
      dtype='object')


In [401]:
# 分割特征和类别
X = dataset_df.iloc[:, :-1]  # 特征
y = dataset_df.iloc[:, -1]   # 类别
X = X.dropna()  # 删除包含缺失值的行

In [425]:
X_train, y_train = dataset_df.loc[:, selected_features], dataset_df.iloc[:, -1]

In [426]:
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)

X = np.array(X_train, dtype=np.float32)
y = np.array(y_train, dtype=np.float32).reshape(-1, 1)

# 转换为 PyTorch 的 Tensor 数据类型
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [427]:
# 定义神经网络模型
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 32)
        self.fc4 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        y = x
        x = self.sigmoid(self.fc4(x))
        return x, y

# 初始化神经网络
net_model = NeuralNetwork()

# 定义损失函数和优化器
criterion = nn.BCELoss()
optimizer = optim.Adam(net_model.parameters(), lr=0.001)

# 将数据转换为 DataLoader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# 训练神经网络
num_epochs = 100
for epoch in range(num_epochs):
    net_model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs, layer = net_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

# 测试神经网络
net_model.eval()
with torch.no_grad():
    y_pred, layer = net_model(X_test)
    y_pred_class = (y_pred >= 0.5).float()
    accuracy = (y_pred_class == y_test).float().mean()
    print(f"Test Accuracy: {accuracy.item():.4f}")

Epoch [1/100], Loss: 0.6860
Epoch [2/100], Loss: 0.4918
Epoch [3/100], Loss: 0.4663
Epoch [4/100], Loss: 0.4625
Epoch [5/100], Loss: 0.4481
Epoch [6/100], Loss: 0.4510
Epoch [7/100], Loss: 0.4638
Epoch [8/100], Loss: 0.4449
Epoch [9/100], Loss: 0.4519
Epoch [10/100], Loss: 0.4502
Epoch [11/100], Loss: 0.4511
Epoch [12/100], Loss: 0.4500
Epoch [13/100], Loss: 0.4466
Epoch [14/100], Loss: 0.4370
Epoch [15/100], Loss: 0.4427
Epoch [16/100], Loss: 0.4283
Epoch [17/100], Loss: 0.4357
Epoch [18/100], Loss: 0.4355
Epoch [19/100], Loss: 0.4306
Epoch [20/100], Loss: 0.4215
Epoch [21/100], Loss: 0.4201
Epoch [22/100], Loss: 0.4214
Epoch [23/100], Loss: 0.4181
Epoch [24/100], Loss: 0.4277
Epoch [25/100], Loss: 0.4260
Epoch [26/100], Loss: 0.4144
Epoch [27/100], Loss: 0.4110
Epoch [28/100], Loss: 0.4062
Epoch [29/100], Loss: 0.4297
Epoch [30/100], Loss: 0.4043
Epoch [31/100], Loss: 0.4026
Epoch [32/100], Loss: 0.4056
Epoch [33/100], Loss: 0.4095
Epoch [34/100], Loss: 0.4153
Epoch [35/100], Loss: 0

In [368]:
params_lgb = {
    'objective': 'binary',  # 二分类目标函数
    'metric': 'binary_logloss',  # 二分类损失函数
    'boosting_type': 'gbdt',  # 提升类型，可选参数：'gbdt', 'dart', 'goss'
    'num_leaves': 28,  # 叶子节点数量
    'learning_rate': 0.1,  # 学习率
    'feature_fraction': 0.85,  # 特征采样比例
    'bagging_fraction': 0.7,  # 数据采样比例
    'bagging_freq': 4,  # 数据采样频率
    'random_state': 42,
    'min_child_samples': 8,
    'verbose':-1, 
    'random_state':12
}
params_cat = {
    'iterations': 626,
    'learning_rate': 0.01,
    'depth': 4,
    'l2_leaf_reg': 0.27,
    'border_count': 66,
    'random_state': 12,
    'verbose': False
}
params_xgb = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'eta': 0.0356,
    'max_depth': 7,
    'subsample': 0.95,
    'colsample_bytree': 0.92,
    'gamma': 9e-08,
    'random_state': 12
}
params_RF = {
    'n_estimators': 300,
    'max_depth': 6,
    'min_samples_split': 2,
    'min_samples_leaf': 3,
    'max_features': 'auto',
    'random_state': 12
}
params_net = {
    'hidden_layer_sizes': (200,50), 
    'activation': 'relu', 
    'solver': 'adam', 
    'alpha': 0.004524225053160557, 
    'batch_size': 'auto', 
    'learning_rate': 'constant', 
    'learning_rate_init': 0.0032695885785495216, 
    'power_t': 0.41229208337868917, 
    'max_iter': 391, 
    'shuffle': True, 
    'random_state': None, 
    'tol': 0.0002505082147304683, 
    'verbose': False, 
    'warm_start': False, 
    'momentum': 0.8862529058691623, 
    'nesterovs_momentum': True, 
    'early_stopping': True, 
    'validation_fraction': 0.1250369526043429,      
    'beta_1': 0.8337047263985674, 
    'beta_2': 0.9945669606589083, 
    'epsilon': 1.479694727954582e-09, 
    'n_iter_no_change': 19, 
    'max_fun': 16800
}

In [369]:
def map_values(value,thred=best_threshold):
    """根据阈值映射"""
    if value <= thred:
        return 0.5 * (value / thred)
    else:
        return 0.5 + 0.5 * ((value - thred) / 0.65)
y_pred = np.vectorize(map_values)(y_pred)

In [380]:
%%time

Models = []
best_acu = 0
best_threshold = 0
# 定义交叉验证的折数
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

# 定义模型列表
models = [
    lgb.LGBMClassifier(**params_lgb),
    #cb.CatBoostClassifier(**params_cat),
    # xgb.XGBClassifier(**params_xgb),
    RandomForestClassifier(**params_RF),
    # MLPClassifier(**params_net)
    # SVC(probability=True, random_state=42)  # 设置 probability=True 以输出概率
]

for train_index, valid_index in kf.split(X_train):
    
    X_tr, X_val = X_train.iloc[train_index], X_train.iloc[valid_index]
    y_tr, y_val = y_train.iloc[train_index], y_train.iloc[valid_index]
    
    # 训练模型
    for model in models:
        model.fit(X_tr, y_tr)

    # 预测结果的概率值
    y_pred_proba_list = []
    for model in models:
#         if type(model).__name__ == 'MLPClassifier':
#             scaler = StandardScaler()
#             features_to_scale = X_val.columns
#             X_val[features_to_scale] = scaler.fit_transform(X_val[features_to_scale])
        y_pred_proba = model.predict_proba(X_val)[:, 1]  # 取第一列的概率值（正类的概率）
        y_pred_proba_list.append(y_pred_proba)
        
#     for i in range(len(y_pred_proba_list)):
#         thresholds = np.linspace(0, 1, 100)
#         acu = 0
#         thred = 0
#         for threshold in thresholds:
#             y_pred = np.where(y_pred_proba_list[i] >= threshold, 1, 0)
#             accuracy = np.mean(y_pred == y_val)
#             if accuracy > acu:
#                 acu = accuracy
#                 thred = threshold
#                 print('acu=', acu)
#         y_pred_proba_list[i] = [map_values(j, thred=thred) for j in y_pred_proba_list[i]]
            

    # 模型融合（平均概率值）
    y_pred_ensemble_proba = np.mean(y_pred_proba_list, axis=0)
    
    #选取最佳阈值
    thresholds = np.linspace(0, 1, 100)
    temp_acu = 0
    temp_thred = 0
    for threshold in thresholds:
        # 计算最佳阈值
        y_pred_ensemble = np.where(y_pred_ensemble_proba >= threshold, 1, 0)
        accuracy = np.mean(y_pred_ensemble == y_val)
        if accuracy >= temp_acu:
            temp_acu = accuracy
            temp_thred = threshold
            temp_y_pred = y_pred_ensemble
    print(classification_report(y_val, temp_y_pred))
    print(roc_auc_score(y_val, y_pred_ensemble_proba))
    
    # 保存最优模型和最优阈值
    if temp_acu >= best_acu:
        Models = models
        best_acu = temp_acu
        best_threshold = temp_thred
print(best_acu)
print(best_threshold)

[LightGBM] [Warning] feature_fraction is set=0.85, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.85
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       101
           1       1.00      0.91      0.95        23

    accuracy                           0.98       124
   macro avg       0.99      0.96      0.97       124
weighted avg       0.98      0.98      0.98       124

0.9741713301764959
[LightGBM] [Warning] feature_fraction is set=0.85, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.85
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0

In [371]:
test_df = pd.read_csv('test.csv',index_col="Id")
# 填充缺失值为特征的均值
for i in range(len(test_df['EJ'])):
    if test_df['EJ'][i] != 'A' and test_df['EJ'][i] != 'B':
        test_df['EJ'][i] = 'A'
test_df['EJ'] = test_df['EJ'].replace({'A':0, 'B':1})
test_df = test_df.fillna(mean_values)
test_df.replace([np.inf], [np.nan], inplace=True)
test_df = test_df.fillna(max_values)
test_df.replace([-np.inf], [np.nan], inplace=True)
test_df = test_df.fillna(min_values)
test_df = test_df.loc[:, selected_features]

C:\Users\GP1\AppData\Local\Temp\ipykernel_21460\952655860.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['EJ'][i] = 'A'


In [372]:
test_df

,AB,AF,AH,AM,AR,AX,AY,BC,BD,BN,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
Id,,,,,,,,,,,,,,,,,,,,,
00eed32682bb,0.0,28688.18766,0.0,38.968552,0.0,0.0,10.315851,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
010ebe33f668,0.0,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02fa521e1838,0.0,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
040e15f562a2,0.0,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
046e85c7cc7f,0.0,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [373]:
y_pred = []
for model in Models:
    y_pred_proba = model.predict_proba(test_df)[:, 1]  # 取第一列的概率值（正类的概率）
    y_pred.append(y_pred_proba)

# 模型融合（平均概率值）
y_pred = np.mean(y_pred, axis=0)
print(y_pred)

[0.4623881  0.45415844 0.45415844 0.45415844 0.45415844]


In [374]:
def map_values(value,thred=best_threshold):
    """根据阈值映射"""
    if value <= thred:
        return 0.5 * (value / thred)
    else:
        return 0.5 + 0.5 * ((value - thred) / 0.65)
y_pred = np.vectorize(map_values)(y_pred)
pred = np.stack((1-y_pred, y_pred), axis=1)
print(pred)

[[0.44734715 0.55265285]
 [0.45367766 0.54632234]
 [0.45367766 0.54632234]
 [0.45367766 0.54632234]
 [0.45367766 0.54632234]]


In [432]:
a =  [[0., 0.], [2., 3.], [4., 5.]]
# 使用zip函数将a列表中的元素重新组合成元组的形式
transposed_a = zip(*a)

# 使用列表解析，将元组中的元素重新组织成所需的形式
result_a = [[item for item in row] for row in transposed_a]
print(result_a)

[[0.0, 2.0, 4.0], [0.0, 3.0, 5.0]]


In [431]:
from sklearn.neural_network import MLPClassifier
X = [[0., 0.], [1., 1.], [1., 1.]]
y = [0, 1, 1]
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X, y)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,
              solver='lbfgs')

In [375]:
# submission = pd.DataFrame()
# submission["Id"] = test_df.index
# submission["class_0"] = 1 - y_pred
# submission["class_1"] = y_pred
# submission.to_csv('submission.csv', index=False)

In [376]:
submission = pd.DataFrame(test["Id"], columns = ["Id"]);
submission["class_0"] = 1 - y_pred
submission["class_1"] = y_pred

submission.to_csv('submission.csv', index = None);
submission_df = pd.read_csv('submission.csv')

In [377]:
submission_df

,Id,class_0,class_1
0,00eed32682bb,0.447347,0.552653
1,010ebe33f668,0.453678,0.546322
2,02fa521e1838,0.453678,0.546322
3,040e15f562a2,0.453678,0.546322
4,046e85c7cc7f,0.453678,0.546322
